[Node 69: Web-Framework Django](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node69.html)

Navigation:

**Next:** [Aufgaben](node70.ipynb) **Up:** [Web-Programmierung und Django](node67.ipynb) **Previous:** [Web-Programmierung](node68.ipynb)

##  Web-Framework Django

Web-Frameworks fassen mehrere Komponenten zusammen:  
* integrierter WSGI-kompatibler Webserver 
* Templating-System (Text- oder XML-basiert) 
* Anbindung an eine Datenbank  

Django ist ein sogenanntes "leichtes" Web-Framework. Eine Django-Webseite besteht aus einem Django-Projekt, das in der Regel aus mehreren Applikationen besteht. Eine Applikation besteht dabei aus 3 Komponenten:  
* Modelle: objektorientierte Repr sentation der persistenten Daten der Anwendung 
* Views: Controller, die Modelle abfragen und Daten mit Hilfe von Templates generieren. Templates bestimmen das Aussehen der Webseite. 
* URL-Mappings: Abbildungen von Teil-URLs auf einzelne Views

![Image architecture](figures/architecture.png "Image architecture")

### Model-Template-View  
*  <font color=#0000ff> **Models :**</font>  What things are 
*  <font color=#0000ff> **Views :**</font>  How things are processed 
*  <font color=#0000ff> **Templates :**</font>  How things are presented  

### Ein Wiki-Service in Django

Als Beispiel für eine Django Applikation erstellen wir im folgenden Schritt für Schritt einen einfachen [Wiki-Service](https://de.wikipedia.org/wiki/Wiki). Da die Schritte aufeinander aufbauen, gibt es ein [git-Repositorium](https://github.com/fuenfundachtzig/PythonkursDjango), das die folgenden Schritte repräsentiert.

Wir beginnen damit, ein neues Django-Projekt aufzusetzen. (Django muss dafür installiert sein.)
```bash
cd $HOME
mkdir djangoprojs
cd djangoprojs
django-admin startproject wikicamp
```

Es werden mehrere Dateien erzeugt:
```bash
laptop:~/djangoprojs$ cd wikicamp/
laptop:~/djangoprojs/wikicamp$ ls
manage.py wikicamp
```

(An dieser Stelle wäre [git-commit 1](https://github.com/fuenfundachtzig/PythonkursDjango/commit/713471c4292c035970a7c8b2a794cc70ca34e672).)

Um die Datenbank anzulegen, verwenden wir
```bash
python3 manage.py migrate
```

Starten Sie den Entwicklungsserver auf Port 9090 mit:
```bash
python3 manage.py runserver 9090
```
und öffnen Sie in einem Webbrowser folgende Adresse: http://localhost:9090/

---
Da passiert noch nicht viel, man muss erstmal noch die Applikations-Templates anlegen mit:
```bash
laptop:~/djangoprojs/wikicamp$ python3 manage.py startapp wiki
laptop:~/djangoprojs/wikicamp$ ls
manage.py wikicamp wiki
```

(Git: commit 2)

Editieren Sie   <font color=#0000e6>wikicamp/settings.py</font>  und ändern Sie folgende Zeilen - damit definieren wir die verwendeten  Applikationen bzw. Django-Module:
```python
....
TIME_ZONE = 'Europe/Berlin'
....
INSTALLED_APPS = (
    ...
    'wiki',
)
```

Editieren Sie die Datei   <font color=#0000e6>wiki/models.py</font> :
```python
from django.db import models
# Create your models here.                                                      
class Page(models.Model):
    name = models.CharField(max_length=20, primary_key=True)
    content = models.TextField(blank=True)
```

Führen Sie folgenden Befehle aus:
```bash
python3 manage.py makemigrations
python3 manage.py migrate
```

Starten Sie den Django Server erneut:
```bash
laptop:~/djangoprojs/wikicamp$ python manage.py runserver 9090
```

Öffnen Sie die Datei   <font color=#0000e6>wikicamp/urls.py</font>  und definieren Sie eine URL-Abbildung:
```python
...
    path('admin/', admin.site.urls),
    re_path(r'^wikicamp/(?P<page_name>[^/]+)/edit/$','wiki.views.edit_page'),
```

---
Den "view" erzeugen wir nun, indem  wir die Datei  <font color=#0000e6>wiki/views.py</font> editieren:
```python
# Create your views here.
from wiki.models import Page
from django.shortcuts import render_to_response
from django.template.context_processors import csrf # https://docs.djangoproject.com/en/3.0/ref/csrf/
def view_page(request, page_name):
    try:
        page = Page.objects.get(pk=page_name)
        # need to add response later        
    except Page.DoesNotExist:
        c = {"page_name": page_name}
        c.update(csrf(request))
        return render_to_response("create.html", c)
```

Ein erneuter Aufruf der URL   <font color=#0000e6>http://localhost:9090/wikicamp/Start/</font>  liefert einen neuen Fehler:   <font color=#ff0000> **'TemplateDoesNotExist at /wikicamp/Start/ create.html'**</font>  

Dieses Template erzeugen wir nun. Erzeugen Sie das Unterverzeichnis    <font color=#0000e6>djangoprojs/wikicamp/templates</font>  und tragen Sie in   <font color=#0000e6>wikicamp/settings.py</font>   folgende Zeile ein:
```python
TEMPLATES = [
    {
        'BACKEND': 'django.template.backends.django.DjangoTemplates',
        'DIRS': ['templates'],
)
```

Erzeugen Sie die Datei   <font color=#0000e6>templates/create.html</font>  mit folgendem Inhalt:
```html
  <head>
    <title>{{page_name}} - Create</title>
  </head>
  <body>
    <h1>{{page_name}}</h1>
    This page does not exist. <a href="/wikicamp/{{page_name}}/edit/">Create?</a>
  </body>
```

(Git: commit 3)

---

Ein erneuter Aufruf der URL   <font color=#0000e6>http://localhost:9090/wikicamp/Start/</font>  liefert nun eine korrekte HTML Seite. Wenn der   <font color=#008000> *Create*</font>-Link geklickt wird, erhält man eine Fehlermeldung über die fehlende Seite (404).

Diese fügen wir nun in   <font color=#0000e6>wiki/views.py</font>  hinzu:
```python
def edit_page(request, page_name):
    try:
        page = Page.objects.get(pk=page_name)
        content	= page.content
    except Page.DoesNotExist:
        content = ""
    c = {"page_name": page_name, "content":content}   
    c.update(csrf(request))
    return render_to_response("edit.html", c)
```

und in wiki/urls.py</font> :
```python
    re_path(r'^wikicamp/(?P[^/]+)/edit/$', views.edit_page),
    re_path(r'^wikicamp/(?P[^/]+)/save/$', views.save_page)
```

Ein erneuter Aufruf von   <font color=#0000e6>http://localhost:9090/wikicamp/Start/edit</font>  liefert eine Fehlermeldung, dass das  Template noch nicht existiert:   <font color=#ff0000> **TemplateDoesNotExist at /wikicamp/Start/edit/**</font>  
 
Wir legen nun das Template   <font color=#0000e6>templates/edit.html</font>  an:
```html
  <head>
    <title>{{page_name}} - Editing</title>
  </head>
  <body>
    <h1>Editing {{page_name}}</h1>
    <form method="post" action="/wikicamp/{{page_name}}/save/">
       {% csrf_token %}
          <textarea name="content" rows="20" cols="60">{{content}}</textarea><br/>
          <input type="submit" values="Save Page"/>
    </form>
  </body>
```

Zum Speichern des Textfeld-Inhalts definieren wir nun die Methode   <font color=#008000> *save_page()*</font>  in   <font color=#0000e6>wiki/views.py</font>
```python
....
from django.http import HttpResponseRedirect
def save_page(request, page_name):
    content = request.POST["content"]
    try:
        page = Page.objects.get(pk=page_name)
        page.content = content
    except Page.DoesNotExist:
        page = Page(name=page_name, content=content)
    page.save()
    return HttpResponseRedirect("/wikicamp/"+ page_name + "/")
```

Jetzt können Seiten angelegt, aber noch nicht angezeigt werden.
(Es gäbe einen Fehler `ValueError: The view wiki.views.view_page didn't return an HttpResponse object. It returned None instead`.) Deshalb komplettieren wir die   <font color=#008000> *view_page()*</font>  Methode:
```python
...
def view_page(request, page_name):
    try:
        page = Page.objects.get(pk=page_name)
        content = page.content
        c = {"page_name": page_name, "content":content}
        c.update(csrf(request))
        return render_to_response("view.html", c)
    except Page.DoesNotExist:
        c = {"page_name": page_name}
        c.update(csrf(request))
        return render_to_response("create.html", c)
...
```

mit dem dazugehörigen Template  <font color=#0000e6>templates/view.html</font>  mit folgendem Inhalt:
```html
  <head>
    <title>{{page_name}}</title>
  </head>
  <body>
    <h1>{{page_name}}</h1>
    {{content}}
    <hr/>
    <a href="/wikicamp/{{page_name}}/edit">Edit	this page?</a>
  </body>
```

(Git: commit 4)

Füllen Sie die Seite   <font color=#008000> *Start*</font>  Start mit dem Text:    <font color=#008000> *Dies ist die Startseite.*</font>  und klicken Sie den   <font color=#008000> *Submit*</font>-Knopf. Die Seite  wird in der Datenbank gespeichert und mit   <font color=#0000e6>templates/view.hmtl</font>  angezeigt. Klicken Sie den Link   <font color=#008000> *Edit this page?*</font>  - es wird erneut der Bearbeitendialog angezeigt.   

Wir haben hiermit einen Prototyp eines Wiki-Systems erzeugt mit Änderungen bzw. Erzeugung der folgenden Dateien:  
*  <font color=#0000e6>wiki/models.py</font> 
*  <font color=#0000e6>wiki/views.py</font> 
*  <font color=#0000e6>wiki.db</font> 
*  <font color=#0000e6>templates/create.html</font> 
*  <font color=#0000e6>templates/view.html</font> 
*  <font color=#0000e6>templates/edit.html</font> 
*  <font color=#0000e6>settings.py</font> 
*  <font color=#0000e6>urls.py</font>  

#### Bonus (I): Die Django Admin Seite

Erzeugen Sie die Datei   <font color=#0000e6>wiki/admin.py</font>  mit folgendem Inhalt, um in der späteren Administrations-Webseite das   <font color=#0000e6>Page</font>-Objekt sichtbar zu machen:
```python
from wiki.models import Page
from django.contrib import admin
admin.site.register(Page)
```

Ein Administrator-Konto kann angelegt werden mit
```bash
python3 manage.py createsuperuser
```

Rufen Sie die Administrator-Seite in einem Webbrowser unter der Adresse http://localhost:9090/admin auf und loggen Sie sich mit dem eben gesetzten Nutzernamen und Passwort ein. Sie haben auf der  Administrator-Seite volle Kontrolle über Ihr Projekt und die Daten-Einträge. 

(Git: commit 5)

#### Bonus (II): Markdown
Um die Wikiseite nun wie gewohnt mit einer vereinfachten Auszeichungssprache zu editieren, muss jetzt noch das   <font color=#0000ff> **markdown**</font>-Modul einfügt werden. Ändern Sie    <font color=#0000e6>wiki/views.py</font>  wie folgt ab:
```python
.......
import markdown
.......
def view_page(request, page_name):
.......
    try:
        page = Page.objects.get(pk=page_name)
        content = page.content
        from django.utils.safestring import mark_safe
        c = {"page_name": page_name, "content": mark_safe(markdown.markdown(content))}
        c.update(csrf(request))
        return render_to_response("view.html", c)
.......
```

 Dies Zeilen bewirken, dass der Text mit   <font color=#0000ff>[Markdown](http://de.wikipedia.org/wiki/Markdown)</font>-Befehlen editiert und dargestellt werden kann.  
 
(Git: commit 6)